In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
from requests import get
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from pandas import DataFrame as df
url="https://www.skoob.com.br/login/"
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [ ]:
#user data
inputUsuario = driver.find_element(by=By.ID, value="UsuarioEmail")
inputUsuario.send_keys("SeuEmail")  # Substitua "SeuEmail" pelo e-mail de login do Skoob
sleep(1)

inputSenha = driver.find_element(by=By.ID, value="UsuarioSenha")
inputSenha.send_keys("SuaSenha")  # Substitua "SuaSenha" pela senha de login do Skoob
sleep(1)

inputSenha.send_keys(Keys.ENTER)
driver.implicitly_wait(5)

linkLivros = driver.find_element(by=By.LINK_TEXT, value="LIDOS")
linkLivros.click()
sleep(1)

In [ ]:
# Lista para armazenar os dados (ano e link do livro)
dados_livros = []

# Anos que você deseja processar
anos = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

for ano in anos:
    # Localiza o elemento pelo valor do atributo 'data-ng-click' para cada ano
    botao_ano = driver.find_element(by=By.CSS_SELECTOR, value=f"a[data-ng-click=\"anoLeitura('books', 8991083, {ano}, 1)\"]")
    
    # Clica no botão
    botao_ano.click()
    
    # Aguarda o carregamento da página
    sleep(2)

    # Extrai o conteúdo da página usando Selenium
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Extrai os links de livros e adiciona à lista de todos os sites
    for link in soup.find_all('a', href=re.compile(r'.*\.html$')):
        full_url = "https://skoob.com.br" + link.get('href')
        dados_livros.append({'Ano de Leitura': ano, 'Link do Livro': full_url})
        
# Cria um DataFrame a partir da lista de dados
df_livros = pd.DataFrame(dados_livros)

# Exibe a lista final de links
print(df_livros)


In [ ]:
# Lista para armazenar os livros, autores, anos, páginas, notas e gêneros extraídos
livros = []

# Loop para acessar cada site e extrair títulos, autores, anos, páginas, notas e gêneros
for index, row in df_livros.iterrows():
    site = row['Link do Livro']
    ano_leitura = row['Ano de Leitura']
    driver.get(site)
    sleep(2)  # Espera a página carregar
    
    # Extrai o conteúdo da página do livro
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Busca pelo título do livro
    titulo_element = soup.find('strong', class_='sidebar-titulo')
    titulo = titulo_element.get_text(strip=True) if titulo_element else 'Título não encontrado'
    
    # Busca pelo autor do livro
    autor_element = soup.find('a', href=re.compile(r'^/autor/\d+-[^/]+$'))
    autor = autor_element.get_text(strip=True) if autor_element else 'Autor não encontrado'
    
    # Busca pelo ano e páginas
    descricao_element = soup.find('div', class_='sidebar-desc')
    descricao_text = descricao_element.get_text(strip=True) if descricao_element else ''
    
    # Usa expressões regulares para encontrar o ano e as páginas
    ano_match = re.search(r'Ano:\s*(\d{4})', descricao_text)
    paginas_match = re.search(r'Páginas:\s*(\d+)', descricao_text)
    
    ano = ano_match.group(1) if ano_match else 'Ano não encontrado'
    paginas = paginas_match.group(1) if paginas_match else 'Páginas não encontradas'
    
    # Busca pela nota (rating)
    nota_element = soup.find('span', class_='rating')
    nota = nota_element.get_text(strip=True) if nota_element else 'Nota não encontrada'
    
    # Busca pelos gêneros
    generos_element = soup.find('span', class_='pg-livro-generos')
    if generos_element:
        # Divide os gêneros por '/' ou ','
        generos = re.split(r'\s*/\s*|\s*,\s*', generos_element.get_text(strip=True))
        generos = str(generos)  # Converte a lista para uma string no formato de lista
    else:
        generos = 'Gêneros não encontrados'
       
     # Busca pela quantidade de pessoas que leram o livro
    leitores_element = soup.find_all('div', class_='bar')
    leitores = 'Quantidade de leitores não encontrada'  # Valor padrão
    
    for elem in leitores_element:
        if 'LERAM' in elem.get_text(strip=True):
            leitores_link = elem.find('a', class_='text_blue')
            leitores = leitores_link.get_text(strip=True) if leitores_link else 'Quantidade de leitores não encontrada'
            break  # Sai do loop se encontrou a quantidade de leitores

    # Busca pela quantidade de pessoas que abandonaram o livro
    abandonos_element = soup.find_all('div', class_='bar')
    abandonos = 'Quantidade de abandonos não encontrada'  # Valor padrão

    for elem in abandonos_element:
    # Verifica se o texto "ABANDONOS" está presente
        if 'ABANDONOS' in elem.get_text(strip=True):
            abandonos_link = elem.find('a', class_='text_blue')
            abandonos = abandonos_link.get_text(strip=True) if abandonos_link else 'Quantidade de abandonos não encontrada'
            break  # Sai do loop após encontrar a quantidade de abandonos
    
# Adiciona as informações extraídas ao DataFrame df_livros
    df_livros.loc[index, 'titulo'] = titulo
    df_livros.loc[index, 'autor'] = autor
    df_livros.loc[index, 'paginas'] = paginas
    df_livros.loc[index, 'nota'] = nota
    df_livros.loc[index, 'generos'] = generos
    df_livros.loc[index, 'leitores'] = leitores
    df_livros.loc[index, 'abandonos'] = abandonos

# Filtra o DataFrame para remover linhas onde o autor é 'Autores'
df_livros = df_livros[df_livros['autor'] != 'Autores']

# Remove a coluna de links
df_livros.drop(columns=['Link do Livro'], inplace=True)

# Exibe o DataFrame final
print(df_livros)


In [ ]:
df_livros.to_csv('meusdadosskoob.csv')